# Earthquake B-Value Report Generator
## COMP41680/COMP47670 Assignment 1

### Task 2: Data Preparation and Analysis

##### Data Preparation

###### Step 1: Extract useful data from json

In [1]:
import json

with open("data/earthquake.json", "r") as json_file:
    data = json.load(json_file)

earthquakes = data["features"]

###### Step 3: Loop earthquakes and extract useful data, parse the country and store in a csv

In [2]:
from datetime import datetime
from util import transform_country_name

earthquake_list = []

for earthquake in earthquakes:
    place = earthquake["properties"]["place"]
    if place is not None and ',' in place:
        country = place.split(',')[-1].strip()
        transformed_country = transform_country_name(country)
        if transformed_country is not None:
            earthquake_list.append({
                "id": earthquake["id"],
                "time": datetime.fromtimestamp(earthquake["properties"]["time"] / 1000).strftime("%Y-%m-%d %H:%M:%S"),
                "mag": earthquake["properties"]["mag"],
                "long": earthquake["geometry"]["coordinates"][0],
                "lat": earthquake["geometry"]["coordinates"][1],
                "place": place,
                "country": transformed_country
            })
        else: # country not in the transform function
            print("Can not transform country: " + country)
    else: # earthquakes far from any country
        print("Skip " + earthquake["id"] + " (" + place + ")")

Skip us6000m4l0 (western Indian-Antarctic Ridge)
Skip us7000ltrn (Kermadec Islands region)
Skip us6000m4ej (South Sandwich Islands region)
Skip us7000ltrg (South Sandwich Islands region)
Skip us6000m4dj (north of Franz Josef Land)
Skip us6000m4d4 (South Sandwich Islands region)
Skip us6000m4c1 (south of the Fiji Islands)


In [3]:
import geopandas as gpd
from ipyleaflet import Map, basemaps, GeoData
from ipywidgets import HTML

# 读取 SHP 文件
world_shp = gpd.read_file('data/map/World_Countries_Generalized.shp')

# 创建地图
m = Map(center=(0, 0), zoom=2, basemap=basemaps.OpenStreetMap.Mapnik)

# 创建 GeoData 对象并添加到地图中
geo_data = GeoData(geo_dataframe=world_shp,
                   style={'color': 'grey', 'opacity': 0.3, 'weight': 1.9, 'dashArray': '9', 'fillOpacity': 1},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name='World Map')
# 定义点击事件处理程序
def handle_click(event, feature, **kwargs):
    # 获取点击的属性信息
    properties = feature['properties']
    # 获取国家名称
    country_name = properties['COUNTRY']
    
    # 创建 HTML 小部件来显示信息
    html = HTML()
    html.value = f'<b>Country:</b> {country_name}'
    
    # 显示 HTML 小部件
    display(html)

# 将点击事件处理程序绑定到地图的点击事件上
geo_data.on_click(handle_click)

m.add_layer(geo_data)

# 显示地图
m

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [4]:
print(world_shp.head())

   FID         COUNTRY ISO     COUNTRYAFF AFF_ISO  SHAPE_Leng  SHAPE_Area  \
0    1     Afghanistan  AF    Afghanistan      AF   50.803210   62.566714   
1    2         Albania  AL        Albania      AL    9.625038    3.073540   
2    3         Algeria  DZ        Algeria      DZ   70.540082  213.513461   
3    4  American Samoa  AS  United States      US    0.600124    0.013720   
4    5         Andorra  AD        Andorra      AD    0.937352    0.055585   

                                            geometry  
0  POLYGON ((61.27655 35.60725, 61.29638 35.62854...  
1  POLYGON ((19.57083 41.68527, 19.58195 41.69569...  
2  POLYGON ((4.60335 36.88791, 4.63555 36.88638, ...  
3  POLYGON ((-170.74390 -14.37555, -170.74942 -14...  
4  POLYGON ((1.44584 42.60194, 1.48653 42.65042, ...  


In [5]:
# %%time
# import urllib.request, json

# url = "https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2014-01-01&endtime=2014-01-31"
# print("GET " + url)
# response = urllib.request.urlopen(url)
# raw_json = response.read().decode("utf-8")
# data = json.loads(raw_json)
# earthquakes = data["features"]

# from datetime import datetime
# from util import transform_country_name

# place_list = []

# for earthquake in earthquakes:
#     place = earthquake["properties"]["place"]
#     if place is not None and ',' in place:
#         country = place.split(',')[-1].strip()
#         transformed_country = transform_country_name(country)
#         if transformed_country is None:
#             if country not in place_list:
#                 place_list.append(country)
        
# place_list.sort()
# print(place_list)